# 실시간 거래 분석: 시뮬레이션 및 리스크 관리

이 노트북은 실시간 거래 환경을 시뮬레이션하고 리스크 관리 메커니즘을 검증합니다.

In [ ]:
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import warnings

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

warnings.filterwarnings("ignore")

# Import from crypto-quant-system
# from src.strategies.momentum import SimpleMomentumStrategy

# Set up visualization
plt.style.use("seaborn-v0_8-darkgrid")
sns.set_palette("husl")

print("✓ 환경 설정 완료")

## 1. 가격 시뮬레이션 (Geometric Brownian Motion)

실시간 거래 환경을 시뮬레이션하기 위해 GBM 모델을 사용합니다.

In [ ]:
# GBM Price Simulator
class PriceSimulator:
    def __init__(self, initial_price, drift=0.0005, volatility=0.02):
        self.price = initial_price
        self.drift = drift
        self.volatility = volatility
        self.prices = [initial_price]
        self.timestamps = [0]

    def simulate(self, time_steps=100, dt=1.0):
        """Simulate price path using GBM"""
        for t in range(1, time_steps + 1):
            dW = np.random.normal(0, np.sqrt(dt))  # noqa: N806
            dP = self.drift * self.price * dt + self.volatility * self.price * dW  # noqa: N806
            self.price += dP
            self.prices.append(max(self.price, 0))  # Prevent negative prices
            self.timestamps.append(t)
        return self.prices


# Initialize simulator
simulator = PriceSimulator(initial_price=50000, drift=0.0005, volatility=0.02)
prices = simulator.simulate(time_steps=250)  # 250 trading days

print(f"초기 가격: {prices[0]:.2f}")
print(f"최종 가격: {prices[-1]:.2f}")
print(f"최고 가격: {max(prices):.2f}")
print(f"최저 가격: {min(prices):.2f}")

## 2. 포트폴리오 및 전략 설정

In [ ]:
# Simple Portfolio Class
class Portfolio:
    def __init__(self, initial_capital=1000000):
        self.cash = initial_capital
        self.positions = {}
        self.equity_history = [initial_capital]
        self.trades = []

    def buy(self, asset, price, quantity):
        """Buy asset"""
        cost = price * quantity
        if cost <= self.cash:
            self.cash -= cost
            if asset not in self.positions:
                self.positions[asset] = 0
            self.positions[asset] += quantity
            self.trades.append(("BUY", asset, price, quantity))
            return True
        return False

    def sell(self, asset, price, quantity):
        """Sell asset"""
        if asset in self.positions and self.positions[asset] >= quantity:
            self.cash += price * quantity
            self.positions[asset] -= quantity
            self.trades.append(("SELL", asset, price, quantity))
            return True
        return False

    def get_total_value(self, current_price, asset="BTC/KRW"):
        """Get total portfolio value"""
        asset_value = self.positions.get(asset, 0) * current_price
        return self.cash + asset_value

    def calculate_metrics(self, prices):
        """Calculate portfolio metrics"""
        values = []
        for price in prices:
            value = self.get_total_value(price)
            values.append(value)

        returns = np.diff(values) / np.array(values[:-1])

        # VaR (95% confidence)
        var_95 = np.percentile(returns, 5)

        # CVaR (Conditional VaR)
        cvar_95 = returns[returns <= var_95].mean()

        # Sharpe Ratio
        sharpe = np.mean(returns) / np.std(returns) * np.sqrt(252)

        # Maximum Drawdown
        running_max = np.maximum.accumulate(values)
        drawdown = (values - running_max) / running_max
        max_dd = np.min(drawdown)

        return {
            "values": values,
            "var_95": var_95,
            "cvar_95": cvar_95,
            "sharpe": sharpe,
            "max_dd": max_dd,
        }


# Initialize portfolio
portfolio = Portfolio(initial_capital=1000000)

print("포트폴리오 초기화 완료")
print(f"초기 자본: {portfolio.get_total_value(prices[0]):,.0f} 원")

## 3. 실시간 거래 시뮬레이션

In [ ]:
# Simple Momentum Strategy
class SimpleMomentumStrategy:
    def __init__(self, lookback=20, threshold=0.02):
        self.lookback = lookback
        self.threshold = threshold
        self.price_history = []

    def generate_signal(self, price):
        """Generate trading signal"""
        self.price_history.append(price)

        if len(self.price_history) < self.lookback:
            return None

        recent_prices = self.price_history[-self.lookback :]
        momentum = (recent_prices[-1] - recent_prices[0]) / recent_prices[0]

        if momentum > self.threshold:
            return "BUY"
        elif momentum < -self.threshold:
            return "SELL"
        return None


# Initialize strategy
strategy = SimpleMomentumStrategy(lookback=20, threshold=0.02)

# Simulation
for _i, price in enumerate(prices):
    signal = strategy.generate_signal(price)

    if signal == "BUY" and portfolio.cash > price * 10:
        portfolio.buy("BTC/KRW", price, 1)
    elif signal == "SELL" and portfolio.positions.get("BTC/KRW", 0) > 0:
        portfolio.sell("BTC/KRW", price, 1)

    portfolio.equity_history.append(portfolio.get_total_value(price))

print(f"총 거래 수: {len(portfolio.trades)}")
print(f"최종 자산: {portfolio.equity_history[-1]:,.0f} 원")
print(f"수익: {(portfolio.equity_history[-1] / portfolio.equity_history[0] - 1):.2%}")

## 4. 리스크 메트릭 계산

In [ ]:
# Calculate metrics
metrics = portfolio.calculate_metrics(prices)

print("포트폴리오 리스크 메트릭")
print("=" * 50)
print(f"Value at Risk (95%): {metrics['var_95']:.4f}")
print(f"Conditional VaR (95%): {metrics['cvar_95']:.4f}")
print(f"Sharpe Ratio: {metrics['sharpe']:.2f}")
print(f"Maximum Drawdown: {metrics['max_dd']:.2%}")
print(f"\n최종 자산: {metrics['values'][-1]:,.0f} 원")
print(f"초기 자산: {metrics['values'][0]:,.0f} 원")
print(f"수익률: {(metrics['values'][-1] / metrics['values'][0] - 1):.2%}")

## 5. 거래 결과 시각화

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Portfolio Value
axes[0, 0].plot(metrics["values"], linewidth=2, label="Portfolio Value")
axes[0, 0].fill_between(range(len(metrics["values"])), metrics["values"], alpha=0.3)
axes[0, 0].set_title("포트폴리오 가치")
axes[0, 0].set_ylabel("가치 (원)")
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Price vs Portfolio
ax2_1 = axes[0, 1]
ax2_1_twin = ax2_1.twinx()
ax2_1.plot(prices, color="blue", linewidth=2, label="BTC Price")
ax2_1_twin.plot(metrics["values"], color="red", linewidth=2, label="Portfolio Value")
ax2_1.set_title("가격 vs 포트폴리오 가치")
ax2_1.set_ylabel("가격 (원)", color="blue")
ax2_1_twin.set_ylabel("포트폴리오 (원)", color="red")
ax2_1.grid(True, alpha=0.3)

# 3. Drawdown
running_max = np.maximum.accumulate(metrics["values"])
drawdown = (np.array(metrics["values"]) - running_max) / running_max
axes[1, 0].fill_between(range(len(drawdown)), drawdown, alpha=0.5, color="red")
axes[1, 0].set_title("낙폭 (Drawdown)")
axes[1, 0].set_ylabel("낙폭")
axes[1, 0].grid(True, alpha=0.3)

# 4. Returns Distribution
returns = np.diff(metrics["values"]) / np.array(metrics["values"][:-1])
axes[1, 1].hist(returns, bins=30, edgecolor="black", alpha=0.7)
axes[1, 1].axvline(
    metrics["var_95"], color="red", linestyle="--", label=f"VaR 95% ({metrics['var_95']:.4f})"
)
axes[1, 1].set_title("수익률 분포")
axes[1, 1].set_xlabel("수익률")
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ 시각화 완료")

## 6. 실시간 거래 전 체크리스트

실제 거래 실행 전에 다음 사항들을 확인하세요:

### 기술 검증
- [ ] 백테스팅 완료 (최소 1년 데이터)
- [ ] Walk-forward 분석 수행
- [ ] 거래소 API 연동 테스트
- [ ] Paper trading으로 1개월 이상 검증
- [ ] 거래 지연 및 슬리피지 테스트

### 리스크 관리
- [ ] 일일 손실 한도 설정 (예: 초기 자본의 2%)
- [ ] 포지션 크기 제한 (예: 한 거래당 초기 자본의 1%)
- [ ] Stop-loss 규칙 설정 (예: -5% 손실)
- [ ] Take-profit 규칙 설정 (예: +10% 수익)
- [ ] 포트폴리오 리밸런싱 주기 설정
- [ ] VaR 및 CVaR 모니터링

### 운영 체계
- [ ] 거래 로깅 시스템 구축
- [ ] 실시간 모니터링 대시보드
- [ ] 긴급 정지(Kill switch) 메커니즘
- [ ] 텔레그램/이메일 알림 설정
- [ ] 일일 리포팅 자동화
- [ ] 주간 성과 검토 프로세스

### 거래소 설정
- [ ] API 키 안전 저장 (환경 변수 사용)
- [ ] 거래소별 제한사항 확인
- [ ] 최소 주문량 확인
- [ ] 수수료율 확인
- [ ] 입출금 제한 확인

### 자본 관리
- [ ] 초기 투자 규모 결정
- [ ] 손익 분리 규칙 설정
- [ ] 자본 추가/인출 정책
- [ ] 세금 계산 방법 결정
- [ ] 거래 기록 보관

## 7. 결론

### 핵심 학습 사항:

1. **시뮬레이션의 중요성**: 실제 거래 전 충분한 시뮬레이션이 필수
2. **리스크 관리**: VaR, CVaR 등 정량적 지표로 리스크 측정
3. **지속적 모니터링**: 거래 수행 중에도 메트릭 추적
4. **감정 제어**: 자동화된 규칙 따르기
5. **성능 분석**: 정기적인 거래 검토 및 개선

### 다음 단계:

1. 실제 거래소에서 Paper Trading 실행
2. 소규모로 실제 거래 시작 (자본의 10% 이하)
3. 성과 모니터링 및 전략 개선
4. 점진적으로 거래량 증가
5. 새로운 자산/전략 추가

⚠️ **주의**: 암호화폐 거래는 높은 위험을 수반합니다. 여유 자금으로만 거래하고, 충분한 학습과 테스트 후에 시작하세요.